## 聊天机器人 第八课

### 流行平台框架与实战

这堂课主要是带大家一起看看国内外一些主流的聊天机器人搭建平台。

### 图灵机器人

link: http://www.tuling123.com/ 

![title](./img/Tulin123.png)

传说中在中文语境下智能度最高的机器人大脑。

#### 用法Demo
1. 注册
2. 创建自己的机器人
3. 连接微信公众号或QQ
4. 社区

### wxBot

图灵机器人很棒，基本帮我们做到了傻瓜式接入主流中文社交软件。

但是，我们可以发现，这里依旧有限制，

官方提供的傻瓜式服务只能接入微信公众号或者QQ，

如果有更多的需求，那我们就得自己动手了。

比如，**将个人微信变成聊天机器人**

这里我们介绍一个很棒的GitHub项目：WxBot（credit: @liuwons）

link: https://github.com/liuwons/wxBot

这是一个是用Python包装Web微信协议实现的微信机器人框架。

换句话说，就是模拟了我们网页登录微信的状态，并通过网页微信协议传输对话。

好，接下来，我们来用wxbot实现一些有趣的功能：

### 群聊天机器人

（转载自：http://blog.csdn.net/tobacco5648/article/details/50802922 ）

实现效果

![title](img/group_chat.png)

有点像谷歌旗下的聊天工具 Allo

![title](img/Google_Allo.png)

简单说就是，在你聊天的时候，可以随时把小机器人给at出来，并且拉出来跟你聊天。

同时，如果调用图灵机器人的服务的话，你可以让它做很多复杂的工作，

比如，找附近的商店啊，查火车时刻表啊，等等。

#### 运行方法

1. 下载wxBot， 安装python的依赖包。
```
pip install requests
pip install pyqrcode
pip install pypng
pip install Pillow
```
2. 在图灵机器人官网注册账号，申请图灵API key
3. 在bot.py文件所在目录下新建conf.ini文件，内容为(key字段内容为申请到的图灵key):
```
[main]
key=1d2678900f734aa0a23734ace8aec5b1
```
4. 最后我们运行bot.py即可

运行之后，你的terminal会跳出一个二维码。

你按照登录网页版微信的方式，扫一下 登录一下。

你的微信就被”托管“了 >.<

In [ ]:
#!/usr/bin/env python
# coding: utf-8

from wxbot import *
import ConfigParser
import json

class TulingWXBot(WXBot):
    # 拿到API Key
    def __init__(self):
        WXBot.__init__(self)

        self.tuling_key = ""
        self.robot_switch = True

        try:
            cf = ConfigParser.ConfigParser()
            cf.read('conf.ini')
            self.tuling_key = cf.get('main', 'key')
        except Exception:
            pass
        print 'tuling_key:', self.tuling_key
    
    # 从图灵API返回自动回复
    def tuling_auto_reply(self, uid, msg):
        if self.tuling_key:
            # API url
            url = "http://www.tuling123.com/openapi/api"
            user_id = uid.replace('@', '')[:30]
            # API request
            body = {'key': self.tuling_key, 'info': msg.encode('utf8'), 'userid': user_id}
            r = requests.post(url, data=body)
            respond = json.loads(r.text)
            result = ''
            # 拿到回复，进行处理
                # 按照API返回的code进行分类
                # 是否是一个link
                # 还是一句话
                # 还是一组(list)回复
            if respond['code'] == 100000:
                result = respond['text'].replace('<br>', '  ')
                result = result.replace(u'\xa0', u' ')
            elif respond['code'] == 200000:
                result = respond['url']
            elif respond['code'] == 302000:
                for k in respond['list']:
                    result = result + u"【" + k['source'] + u"】 " +\
                        k['article'] + "\t" + k['detailurl'] + "\n"
            else:
                result = respond['text'].replace('<br>', '  ')
                result = result.replace(u'\xa0', u' ')

            print '    ROBOT:', result
            return result
        # 加了个exception，如果没有图灵API Key的话，
        # 那就无脑回知道了。
        else:
            return u"知道啦"
    
    # 如果，用户不想机器人继续BB了，
    # 或者，用户想言语调出机器人：
    def auto_switch(self, msg):
        msg_data = msg['content']['data']
        stop_cmd = [u'退下', u'走开', u'关闭', u'关掉', u'休息', u'滚开']
        start_cmd = [u'出来', u'启动', u'工作']
        if self.robot_switch:
            for i in stop_cmd:
                if i == msg_data:
                    self.robot_switch = False
                    self.send_msg_by_uid(u'[Robot]' + u'机器人已关闭！', msg['to_user_id'])
        else:
            for i in start_cmd:
                if i == msg_data:
                    self.robot_switch = True
                    self.send_msg_by_uid(u'[Robot]' + u'机器人已开启！', msg['to_user_id'])
    
    # 从微信回复
    def handle_msg_all(self, msg):
        if not self.robot_switch and msg['msg_type_id'] != 1:
            return
        if msg['msg_type_id'] == 1 and msg['content']['type'] == 0:  # reply to self
            self.auto_switch(msg)
        elif msg['msg_type_id'] == 4 and msg['content']['type'] == 0:  # text message from contact
            self.send_msg_by_uid(self.tuling_auto_reply(msg['user']['id'], msg['content']['data']), msg['user']['id'])
        elif msg['msg_type_id'] == 3 and msg['content']['type'] == 0:  # group text message
            if 'detail' in msg['content']:
                my_names = self.get_group_member_name(msg['user']['id'], self.my_account['UserName'])
                if my_names is None:
                    my_names = {}
                if 'NickName' in self.my_account and self.my_account['NickName']:
                    my_names['nickname2'] = self.my_account['NickName']
                if 'RemarkName' in self.my_account and self.my_account['RemarkName']:
                    my_names['remark_name2'] = self.my_account['RemarkName']

                is_at_me = False
                for detail in msg['content']['detail']:
                    if detail['type'] == 'at':
                        for k in my_names:
                            if my_names[k] and my_names[k] == detail['value']:
                                is_at_me = True
                                break
                if is_at_me:
                    src_name = msg['content']['user']['name']
                    reply = 'to ' + src_name + ': '
                    if msg['content']['type'] == 0:  # text message
                        reply += self.tuling_auto_reply(msg['content']['user']['id'], msg['content']['desc'])
                    else:
                        reply += u"对不起，只认字，其他杂七杂八的我都不认识，,,Ծ‸Ծ,,"
                    self.send_msg_by_uid(reply, msg['user']['id'])


def main():
    bot = TulingWXBot()
    bot.DEBUG = True
    bot.conf['qr'] = 'png'

    bot.run()


if __name__ == '__main__':
    main()

GitHub上还有不少相似的项目，大家都可以关注一下：

[feit/Weixinbot](https://github.com/feit/Weixinbot) Nodejs 封装网页版微信的接口，可编程控制微信消息

[littlecodersh/ItChat](https://github.com/littlecodersh/ItChat) 微信个人号接口、微信机器人及命令行微信，Command line talks through Wechat

[Urinx/WeixinBot](https://github.com/Urinx/WeixinBot) 网页版微信API，包含终端版微信及微信机器人

[zixia/wechaty](https://github.com/zixia/wechaty) Wechaty is wechat for bot in Javascript(ES6). It's a Personal Account Robot Framework/Library.

[WxbotManage](https://coding.net/u/vivre/p/WxbotManage/git) 基于Wxbot的微信多开管理和Webapi系统


### 自定义API接口

刚刚我们讲的部分，还都是调用图灵机器人的API。

我们来看看，如何使用自己的ChatBot模型。

我们这里用之前讲过的Chatterbot库做个例子。

思路还是一样。我们用WxBot来处理微信端的工作，

然后，我们架设一个API，来把chatterbot的回复给传到微信去。

这里我们用一个简单粗暴的API框架：Hug。

大家也可以用其他各种框架，比如Flask。

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# 导入chatterbot自带的语料库
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer
import hug

deepThought = ChatBot("deepThought")
deepThought.set_trainer(ChatterBotCorpusTrainer)
# 使用中文语料库训练它
deepThought.train("chatterbot.corpus.chinese")  # 语料库

# API框架
@hug.get()
def get_response(user_input):
    response = deepThought.get_response(user_input).text
    return {"response":response}

hug -f bot_api.py

跑起来以后，你的terminal大概长这样：

![title](img/hug.png)

于是你可以在浏览器中尝试：

![title](img/test.png)

好，

接下来的部分，依旧是感谢@liuwons的wxbot：

In [ ]:
#!/usr/bin/env python
# coding: utf-8

from wxbot import WXBot
import requests

# base url
bot_api="http://127.0.0.1:8000/get_response"

# 处理回复
class MyWXBot(WXBot):
    def handle_msg_all(self, msg):
        if msg['msg_type_id'] == 4 and msg['content']['type'] == 0:
            user_input = msg["content"]["data"]
            payload={"user_input":user_input}
            response = requests.get(bot_api,params=payload).json()["response"]
            self.send_msg_by_uid(response, msg['user']['id'])

def main():
    bot = MyWXBot()
    bot.DEBUG = True
    bot.conf['qr'] = 'png'
    bot.run()

if __name__ == '__main__':
    main()

聊起来大概是这个节奏。

![title](img/screen_shot.png)

好，如此，我们能做的显然不仅仅是用了个chatterbot

因为我们这样的一个结构，我们其实可以把任何聊天模型应用进来。

简单那来说，

Model <--API--> WxBot <--web--> WeChat

我们要做的就是封装一下我们的模型，让它看到一句话，给一个回复。

或者甚至是把VQA加到微信机器人中，发图片再发问题，求回复~

### 一些有趣的微信机器人的idea

* 跨群转发。这是个非常实用的功能。对群来说，因为微信一个群最多500人， 跨群转发可以有效地把两个群拼到一起，实现更广泛的讨论。对个人来说，也可以用有选择的转发来把信息归档。比如看老板或者妹子在你加的几个群里每天都说了啥等等。 
* 聊天消息的主题归并，分析和搜索。微信聊天的基本单位是消息，但消息本身是非常碎片化的，很不适合搜索和分析。机器人可以把相关主题的消息归并起来，一方面可以大幅减小信息过载，一方面也可以从中得到更有价值的信息（类似视频分析里面把帧变成镜头）。这样分析以后可以做知识归档，用OneNote/印象笔记甚至公众号把讨论的成果沉淀下来。 
* 聊天脉络的梳理。群里的人一多，经常会出现几个话题并行出现的情况。这种情况对于理解和搜索都是非常不利的。机器人也需要把聊天的脉络进行梳理，在同一时间，把不同主题分别开。 
* 基本的统计数据。比如发言时间的分布，群的活跃度，成员的活跃度等等。做成漂亮的可视化，用户应该也会喜欢，给产品加分。 


### 国际上比较主流的几大聊天机器人框架

* wit.ai
* api.ai
* microsoft bot framework

虽然...

但是我们还是要稍微了解一下行业内的一些insights，

看看现在业内大火的各种炒聊天机器人概念的startups都是怎么玩的。

我们选点简单的开始玩起。

机器人端：api.ai，背后是谷歌

聊天端，我们选个Telegram，主打安全和快速的画风清奇的聊天软件~

#### Demo

1. 注册api.ai
2. 创建api.ai机器人
3. 设置intents, entities等等
4. 及时测试
5. 走起telegram
6. 跟机器人爸爸聊天, 设置

7. ![title](img/botfather.png)

8. integrate
9. 托管GitHub, Heroku
10. 手机端试试玩儿

11. ![title](img/julyedu_demo.png)

### 直接调用api.ai进你自己的app/平台

其实这几个大平台都有自己的各个语言的官方支持库，让生活变得灰常简单：

In [ ]:
!/usr/bin/env python
 -*- coding: utf-8 -*-

import os.path
import sys
import json
try:
    import apiai
except ImportError:
    sys.path.append(
        os.path.join(os.path.dirname(os.path.realpath(__file__)), os.pardir)
    )
    import apiai
    
# api token
CLIENT_ACCESS_TOKEN = 'your client access token'

def main():
    while(1):
        ai = apiai.ApiAI(CLIENT_ACCESS_TOKEN)
        request = ai.text_request()
        request.lang = 'en'  # 中文英文法语西语等等各种

        # request.session_id = "<SESSION ID, UBIQUE FOR EACH USER>"
        print("\n\nYour Input : ",end=" ")
        request.query = input()

        print("\n\nBot\'s response :",end=" ")
        response = request.getresponse()
        responsestr = response.read().decode('utf-8')
        response_obj = json.loads(responsestr)

        print(response_obj["result"]["fulfillment"]["speech"])

if __name__ == '__main__':
    main()

跟之前的思路其实都差不多，

就是用框架的API调用来传送『对话』，解决问题

:P

就是这样！